In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import traceback
import pickle
import numpy as np
from utils.data_handling import load_data, save_model_to_db
from utils.explainer import generate_shap_explainer
from preprocessing.preprocessing import label_target, impute_null_features, label_features, scale_features, select_features
from classification.param_search import optimize_hyperparameters_with_random_search
from classification.evaluation import evaluate_model
from classification.train import train_model
from loggers import logger

In [ ]:
source_type = 'csv'
input_path = './train.csv'
target_col = 'Survived'
model_name = 'lgb'

In [ ]:
# 데이터 불러오기
data = load_data(source_type=source_type, input_path=input_path)

# 타겟 컬럼 labeling
data, target_dict = label_target(data=data, target_col=target_col)

# 필요없는 컬럼 제거
# TODO: 데이터마다 바뀜
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, errors='ignore', inplace=True)

# null 값 impute
data, null_converter = impute_null_features(data=data)

# 파생변수 생성
# TODO: 데이터마다 바뀜
logger.info('## Create derived variables')
data["Fsize"] = data["SibSp"] + data["Parch"] + 1
# 파생변수 생성 후 필요없는 컬럼 제거
data = data.drop(["SibSp", "Parch"], axis=1)
cols = list(data.columns.difference(['target'])) + ['target']
data = data[cols]
logger.info('data: \n %s' % list(data.columns.difference(['target'])))

# categorical features 파악
# TODO: 데이터에 따라 직접 지정
categorical_feats_name = [col for col in data.columns if data[col].dtype == 'object']

# feature 컬럼 labeling
# lightgbm 이 categorical feats 도 int, float 형식으로 다 바꿔야 작동하므로 일괄 적용
data, label_encoder = label_features(data)

# scaling
if model_name in ['rf', 'xgb', 'lgb', 'catboost']:
    pass
else:
    data = scale_features(data)

# feature selection
features_selected = select_features(data=data, categorical_feats_name=categorical_feats_name)

# TODO: 데이터에 따라 직접 지정
cols = features_selected + ['target']
data = data[cols]
categorical_feats = [idx for idx, col in enumerate(data.columns) if col in categorical_feats_name]

In [ ]:
categorical_feats

In [ ]:
X = data[features_selected].values
y = data['target'].values

In [ ]:
data

In [ ]:
X

In [ ]:
y

In [ ]:
vif

In [ ]:
categorical_feats

In [ ]:
cols = list(data.columns.difference(['target'])) + ['target']

In [ ]:
data

In [ ]:
data[cols]

In [ ]:
data

In [ ]:
X

In [ ]:
data.columns

In [ ]:
data.iloc[:, [1,4]]

In [ ]:
X[:, [1,4]]

In [ ]:
X.type

#### Catboost 예제

In [ ]:
from catboost import CatBoostRegressor

cat_features = [0, 1, 2]
train_data = [["a", "b", 1, 4, 5, 6], ["a", "b", 4, 5, 6, 7]]

train_labels = [10, 20]

model = CatBoostRegressor(iterations=2, learning_rate=1, depth=2)

model.fit(train_data, train_labels, cat_features)

In [ ]:
model.predict(train_data)

#### AWS test

In [ ]:
import pickle

In [ ]:
with open('qq.pkl', 'rb') as f:
    content = pickle.load(f)

In [ ]:
content

In [ ]:
content['fitted_model']

In [ ]:
content['explainer']

In [ ]:
's3://ml-streamlit/model_objects.pkl'

In [ ]:
import os

os.getcwd()

In [ ]:
with open('qqqq.pkl', 'wb') as handle:
    pickle.dump(content['fitted_model'], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('qqqq.pkl', 'rb') as handle:
    result = pickle.load(handle)

In [ ]:
result

In [ ]:
fitted_model

In [ ]:
with open('sample.pkl', 'rb') as f:
    a = pickle.load(f)

In [ ]:
a['fitted_model']

In [ ]:
import boto3

os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-2'  # 서울 리전
os.environ['AWS_PROFILE'] = "ml-streamlit"

s3 = boto3.client('s3')
with open('asdf', 'wb') as f:
    s3.download_fileobj('ml-streamlit', 'model_objects.pkl', f)

In [ ]:
s3.download_file('ml-streamlit', 'model_objects.pkl', 'qq.pkl')

In [ ]:
with open('asdf', 'rb') as f:
    c = pickle.load(f)

In [ ]:
c

In [ ]:
from utils.data_handling import read_from_s3, write_to_s3

In [ ]:
model_objects = read_from_s3('s3://ml-streamlit/model_objects.pkl')

In [ ]:
model_objects

In [ ]:
write_to_s3(model_objects, 's3://ml-streamlit/model_objects_99.pkl')

In [ ]:
aaa = read_from_s3('s3://ml-streamlit/model_objects_99.pkl')

In [ ]:
aaa

In [ ]:
import pymongo
import os

DB_USERNAME = os.environ.get('MONGO_USERNAME', 'root')
DB_PASSWORD = os.environ.get('MONGO_PASSWORD', 'root')
DB_HOST = os.environ.get('MONGO_HOST', 'localhost')
DB_PORT = os.environ.get('MONGO_PORT', '27017')
CONN_STR = "mongodb://%s:%s@%s:%s" % (DB_USERNAME, DB_PASSWORD, DB_HOST, DB_PORT)
DB_DATABASE_NAME = os.environ.get('MONGO_INITDB_DATABASE', 'ml')
DB_MODEL_COLLECTION_NAME = os.environ.get('MONGO_MODEL_COLLECTION_NAME', 'model')
DB_DATA_COLLECTION_NAME = os.environ.get('MONGO_DATA_COLLECTION_NAME', 'data')

client = pymongo.MongoClient(CONN_STR)
database = client[DB_DATABASE_NAME]
collection = database[DB_MODEL_COLLECTION_NAME]

cursor = collection.find()
result = list(cursor)

In [ ]:
result

In [ ]:
import pickle

with open('model_objects.pkl', 'rb') as f:
    c = pickle.load(f)

In [ ]:
c